In [2]:
#Import dependencies
import pandas as pd
import numpy as np

#File paths for csv's

file_path_schools = "PyCitySchools/Resources/schools_complete.csv"
file_path_students = "PyCitySchools/Resources/students_complete.csv"

#Read csv's
schools_data = pd.read_csv(file_path_schools)
students_data = pd.read_csv(file_path_students)

#Seeing what they have in common
schools_data.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [3]:
#Seeing what they have in common
students_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


## Setting up the dataframe

In [5]:
#Merging the dataframes
school_data_merged = pd.merge(students_data, schools_data, how="left", on=["school_name", "school_name"])
school_data_merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [33]:
school_data_merged = school_data_merged.rename({"grade":"Grade", 
                                               "reading_score":'Reading Score', "math_score":"Math Score",
                                              'type':"School Type", 'size': "School Size"})
school_data_merged.columns

Index(['Student ID', 'student_name', 'gender', 'Grade', 'school_name',
       'Reading Score', 'Math Score', 'School ID', 'School Type',
       'School Size', 'budget'],
      dtype='object')

## District Summary

In [34]:
#Calculate the total number of schools
tot_schools = school_data_merged['school_name'].nunique()

In [38]:
#Calculate the total number of students
tot_students = '{:,}'.format(school_data_merged['student_name'].count())

In [58]:
tot_budget = '${:,.2f}'.format(school_data_merged['budget'].unique().sum())

In [47]:
#Calculate the average math score
avg_math_score = school_data_merged['Math Score'].mean()

In [42]:
#Calculate the average reading score
avg_reading_score = school_data_merged['Reading Score'].mean()

In [48]:
#Calculate the overall passing rate (overall average score)
overall_pass_rate = (avg_math_score + avg_reading_score) / 2

In [44]:
#Calculate the percentage of students with a passing math score (70 or greater)
pass_rate_math = (len(school_data_merged[school_data_merged['Math Score'] >= 70]) / school_data_merged.shape[0]) * 100

In [46]:
#Calculate the percentage of students with a passing reading score (70 or greater)
pass_rate_read = (len(school_data_merged[school_data_merged['Reading Score'] >= 70]) / school_data_merged.shape[0])*100

In [57]:
district = (tot_schools, tot_students, tot_budget,
            avg_math_score, avg_reading_score, pass_rate_math,
            pass_rate_read, overall_pass_rate)
District_Summary = pd.DataFrame([district], columns= ['Total Schools', 'Total Students', 'Total Budget', 'Average Math Score', 'Average Reading Score', '% Passing Math',
            '% Passing Reading','% Overall Passing Rate'])
District_Summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606
